In [27]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

!pip install transformers
from transformers import *
import os
import sys
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import numpy as np
import re
import pickle
import time
import pandas as pd
from pathlib import Path
import random
from torch.utils.tensorboard import SummaryWriter

import datetime
import pytz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
!pip install gdown
import gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
dir_path = '/content/drive/MyDrive/qia-hackathon2023'
id = '1JsfTc5uR5ubR2aHUVjFqf2hNzwA8_EZJ'

if os.path.exists(dir_path):
  print("Directory already exists")
else:
  os.chdir('/content/drive/MyDrive')
  !gdown https://drive.google.com/uc?id={id}
  !unzip /content/drive/MyDrive/qia-hackathon2023.zip -d /content/drive/MyDrive
  !rm -rf /content/drive/MyDrive/qia-hackathon2023.zip

Directory already exists


In [57]:
korea_tz = pytz.timezone('Asia/Seoul')
now = datetime.datetime.now(tz = korea_tz)
now = now.strftime("%m_%d_%H")
print(now)

05_24_17


In [58]:
# make new date directory
dir_path = f'/content/drive/MyDrive/qia-hackathon2023/data/{now}'
try:
    os.mkdir(dir_path)
    print("Directory created successfully")
except OSError as error:
    print("Directory creation failed:", error)

Directory created successfully


In [59]:
# load datasets
df = pd.read_csv('/content/drive/MyDrive/qia-hackathon2023/data/hackathon_train.csv', encoding = 'cp949', index_col = 0)

df.tail()

,User_ID,Gender,Age,MBTI,Q_number,Answer
Data_ID,,,,,,
14384,240,0,40,ISTJ,44,<그렇다> 저는 계획에 차질이 생기면 돌아가기 위해 노력을 합니다. 이유는 그 계획...
14385,240,0,40,ISTJ,45,<그렇다> 저는 예전의 실수를 후회할 때가 많습니다. 이유는 그만큼 나태하게 산 적...
14386,240,0,40,ISTJ,46,<아니다> 저는 인간의 존재와 삶의 이유에 대해 깊이 생각하지 않습니다. 이유는 이...
14387,240,0,40,ISTJ,47,<아니다> 저는 감정에 휘둘리는 편이 아닙니다. 이유는 감정을 감추고 밖으로 표현하...
14388,240,0,40,ISTJ,48,<아니다> 저는 상대방 잘못일 때 상대방의 체면을 살려주기 위해 노력하지 않습니다....


In [60]:
# load question set
question = pd.read_csv('/content/drive/MyDrive/qia-hackathon2023/data/Question.csv', encoding = 'utf-8', index_col = 0)

question.tail()

,index.1,Question
index,,
56,56,단계를 건너뛰는 일 없이 절차대로 일을 완수하는 편인가요? 그러한 최근 경험은 어떤...
57,57,논란이 되거나 논쟁을 불러일으킬 수 있는 주제에 관심이 많나요? 최근의 사례를 말씀...
58,58,자신보다 다른 사람에게 더 필요한 기회라고 생각되면 기회를 포기할 수도 있나요? 이...
59,59,마감 기한을 지키기가 힘든가요? 경험을 이야기해보아요.
60,60,일이 원하는 대로 진행될 것이라는 자신감이 있나요? 그렇게 된 계기나 이유가 있나요.


In [61]:
df.isnull().any()

User_ID     False
Gender      False
Age         False
MBTI        False
Q_number    False
Answer      False
dtype: bool

In [62]:
# load pretrained model
def get_model():
  # * Model          | Tokenizer          | Pretrained weights shortcut
  # MODEL=(DistilBertModel, DistilBertTokenizer, 'distilbert-base-uncased')
  tokenizer = BertTokenizerFast.from_pretrained("klue/bert-base")
  model = BertModel.from_pretrained("klue/bert-base")
  n_hl = model.config.num_hidden_layers
  # embed_dim = model.config.embedding_size
  # return model, tokenizer, n_hl, embed_dim
  return model, tokenizer, n_hl

In [63]:
# model, tokenizer, n_hl, embed_dim = get_model()
model, tokenizer, n_hl = get_model()

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "a

In [64]:
# split train and test dataframe
train_df_list = []
test_df_list = []
for idx in df['User_ID'].unique():
    train_df_list.append(df[df['User_ID']==idx][0:40])
    test_df_list.append(df[df['User_ID']==idx][40:])

In [65]:
train_df = pd.concat(train_df_list, ignore_index = True)
test_df = pd.concat(test_df_list, ignore_index = True)

In [66]:
train_df

,User_ID,Gender,Age,MBTI,Q_number,Answer
0,1,1,30,INFP,1,<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...
1,1,1,30,INFP,2,<중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...
2,1,1,30,INFP,3,<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...
3,1,1,30,INFP,4,<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...
4,1,1,30,INFP,5,<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...
...,...,...,...,...,...,...
9595,240,0,40,ISTJ,36,<아니다> 저는 즐거운 파티나 행사로 일주일 피로를 푸는 편이 아닙니다. 이유는 그...
9596,240,0,40,ISTJ,37,<중립> 저는 미술관 가는 일을 좋아하지 않습니다. 이유는 미술 작품에 관심이 많이...
9597,240,0,40,ISTJ,38,<그렇다> 저는 다른 사람의 감정을 이해하기 힘들 때가 많습니다. 이유는 각자의 살...
9598,240,0,40,ISTJ,39,<아니다> 저는 매일 할 일을 계획하지 않습니다. 이유는 매일매일보다는 크게 한 건...


In [67]:
# Concatenate the 'Question', 'Answer', 'Age' and 'Gender' columns
train_texts = [str([train_df['Q_number'][i]]) + ' ' + str(train_df['Answer'][i]) + ' ' + str(train_df['Age'][i]) + ' ' + str(train_df['Gender'][i]) for i in range(len(train_df))]
test_texts = [str([train_df['Q_number'][i]]) + ' ' + str(test_df['Answer'][i]) + ' ' + str(test_df['Age'][i]) + ' ' + str(test_df['Gender'][i]) for i in range(len(test_df))]

# Tokenize the concatenated texts
train_tensor = tokenizer(train_texts, max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)
test_tensor = tokenizer(test_texts, max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [68]:
class MyMapDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        data = {k:v[idx] for k,v in self.data.items()}
        return data

In [69]:
train_ds = MyMapDataset(train_tensor)
test_ds = MyMapDataset(test_tensor)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=False)

In [70]:
def forward(model, dl, device=0):
    pooled = []
    hidden = []
    model.cuda(device)
    model.eval()
    for data in dl:
        data = {k:v.cuda(device) for k,v in data.items()}
        with torch.no_grad():
            output = model(**data, output_hidden_states=True)
        p, h = output.pooler_output, output.hidden_states
        pooled.append(p) # pooler output
        hidden.append(h[-1][:,0,:]) # only [CLS] token embedding 
    return torch.cat(pooled), torch.cat(hidden)

In [71]:
train_result = forward(model, train_dl, device=0)
test_result = forward(model, test_dl, device=0)

In [72]:
train_result[0].shape

torch.Size([9600, 768])

In [73]:
train_result[1].shape

torch.Size([9600, 768])

In [74]:
train_df.head()

,User_ID,Gender,Age,MBTI,Q_number,Answer
0,1,1,30,INFP,1,<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...
1,1,1,30,INFP,2,<중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...
2,1,1,30,INFP,3,<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...
3,1,1,30,INFP,4,<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...
4,1,1,30,INFP,5,<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...


In [75]:
torch.save(train_result, f'/content/drive/MyDrive/qia-hackathon2023/data/{now}/train_embed.pt')
torch.save(test_result, f'/content/drive/MyDrive/qia-hackathon2023/data/{now}/test_embed.pt')

Train

In [76]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from transformers import *
import os
import sys
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import numpy as np
import re
import pickle
import time
import pandas as pd
from pathlib import Path
import random
from torch.utils.tensorboard import SummaryWriter

In [77]:
# load datasets
df = pd.read_csv('/content/drive/MyDrive/qia-hackathon2023/data/hackathon_train.csv', encoding = 'cp949', index_col = 0)

# split train and test dataframe
train_df_list = []
test_df_list = []
for idx in df['User_ID'].unique():
    train_df_list.append(df[df['User_ID']==idx][0:40])
    test_df_list.append(df[df['User_ID']==idx][40:])
    
train_df = pd.concat(train_df_list, ignore_index=True)
test_df = pd.concat(test_df_list, ignore_index=True)

In [78]:
# load embedding
train_result = torch.load(f'/content/drive/MyDrive/qia-hackathon2023/data/{now}/train_embed.pt')
test_result = torch.load(f'/content/drive/MyDrive/qia-hackathon2023/data/{now}/test_embed.pt')

In [79]:
def set_random(SEED=0):
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

class MyDataset(Dataset):
    def __init__(self, data, label, label_idx=0):
        self.data = data
        self.label = label
        self.label_idx = label_idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], torch.tensor(self.label[idx][self.label_idx])
    
def convert_mbti_to_label(mbti: str):
    """
    :param mbti: string. length=4
    :return:
    """
    stand = 'ISTJ'  # [0, 0, 0, 0]
    result = []
    for i in range(4):
        if stand[i] == mbti[i]:
            result.append(0)
        else:
            result.append(1)

    return result

In [80]:
def train(model, dl, optimizer, criterion, device=1):
    model = model.cuda(device)
    model.train()
    loss_all, acc_all = 0, 0
    
    for x, y in dl:
        x, y = x.cuda(device), y.cuda(device)
        output = model(x)
        loss = criterion(output, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(axis=1) == y).sum() / len(y)

        loss_all += loss.item()
        acc_all += acc.item()


    loss = loss_all / len(dl)
    acc = acc_all / len(dl)

    return loss, acc

def valid(model, dl, optimizer=None, criterion=None, device=1):
    model = model.cuda(device)
    model.eval()
    loss_all, acc_all = 0, 0
    
    output_list = []
    for x, y in dl:
        x, y = x.cuda(device), y.cuda(device)
        output = model(x)
        loss = criterion(output, y)

        acc = (output.argmax(axis=1) == y).sum() / len(y)

        loss_all += loss.item()
        acc_all += acc.item()

        output_list.append(output.argmax(dim=1).cpu())
        
    loss = loss_all / len(dl)
    acc = acc_all / len(dl)

    return loss, acc

In [81]:
def forward(model, dl, device=0):
    pooled = []
    hidden = []
    model.cuda(device)
    model.eval()
    for data in dl:
        data = {k:v.cuda(device) for k,v in data.items()}
        with torch.no_grad():
            output = model(**data, output_hidden_states=True)
        p, h = output.pooler_output, output.hidden_states
        pooled.append(p) # pooler output
        hidden.append(h[-1][:,0,:]) # only [CLS] token embedding 
    return torch.cat(pooled), torch.cat(hidden)

In [82]:
def main(label_idx=0, device=1, name='test'):
    
    model = nn.Sequential(nn.Linear(768, 50),
                              nn.ReLU(),
                              nn.Linear(50, 2))
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
    
    
    # dataset / dataloader
    train_data = train_result[0] # pooler output
    train_label = train_df['MBTI'].map(convert_mbti_to_label)
    
    test_data = test_result[0]
    test_label = test_df['MBTI'].map(convert_mbti_to_label)
    
    train_ds = MyDataset(train_data, train_label, label_idx)
    test_ds = MyDataset(test_data, test_label, label_idx)

    train_dl = DataLoader(train_ds, batch_size=1024, shuffle=True)
    test_dl = DataLoader(test_ds, batch_size=1024, shuffle=False)

    # train
    train_final = []
    val_final = []

    # max_val_acc
    max_val_acc_epoch = 0
    max_val_acc = 0
    
    save_dir = f'/content/drive/MyDrive/qia-hackathon2023/data/{now}/ckpt/{name}'
    for epoch in range(1000):
        train_loss, train_acc = train(model, train_dl, optimizer, criterion, device=device)
        # validation
        val_loss, val_acc = valid(model, test_dl, criterion=criterion, device=device)

        # max val_acc
        if(val_acc > max_val_acc): 
          max_val_acc = val_acc
          max_val_acc_epoch = epoch

        writer.add_scalar('Loss/Train', train_loss, epoch)
        writer.add_scalar('Acc/Train', train_acc, epoch)
#         wandb.log({'train_loss': train_loss, 'train_acc': train_acc, 'epoch': epoch})
        writer.add_scalar('Loss/Test', val_loss, epoch)
        writer.add_scalar('Acc/Test', val_acc, epoch)
#         writer.add_scalar('Acc/userid', acc, epoch)
#         wandb.log({'val_loss': val_loss, 'val_acc': val_acc, 'epoch': epoch})

        train_final.append([train_loss, train_acc])
        val_final.append([val_loss, val_acc])
        
        os.makedirs(save_dir, exist_ok=True)
        torch.save(model, f"{save_dir}/epoch_{epoch}.pt")

    return train_final, val_final, max_val_acc, max_val_acc_epoch

In [83]:
# Train all
MBTI = ['IE', 'SN', 'TF', 'JP']
# max_val_acc_epoch list
MBTI_epoch_list = []

set_random(422)
for i in range(4):
    writer = SummaryWriter(f'/content/drive/MyDrive/qia-hackathon2023/data/{now}/tensorboard/test1/{MBTI[i]}/')
    # result = main(i, 0, MBTI[i])
    train_final, val_final, max_val_acc, max_val_acc_epoch = main(i, 0, MBTI[i])
    print(f"{MBTI[i]} max_val_acc is {max_val_acc}, max_val_acc_epoch is {max_val_acc_epoch}")
    MBTI_epoch_list.append(max_val_acc_epoch)

IE max_val_acc is 0.6785714328289032, max_val_acc_epoch is 185
SN max_val_acc is 0.6277901828289032, max_val_acc_epoch is 193
TF max_val_acc is 0.6634347140789032, max_val_acc_epoch is 116
JP max_val_acc is 0.634068101644516, max_val_acc_epoch is 739


Test

In [84]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from transformers import *
import os
import sys
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import numpy as np
import re
import pickle
import time
import pandas as pd
from pathlib import Path
import random
from torch.utils.tensorboard import SummaryWriter

In [85]:
# load datasets
df = pd.read_csv('/content/drive/MyDrive/qia-hackathon2023/data/hackathon_test_for_user.csv', encoding = 'cp949')

In [86]:
df

,Gender,Age,Q_number,Answer
0,0,30,59,<아니다> 저는 모든 일은 정해진 시간을 지켜서 해야 된다고 생각되어서 마감 기한을...
1,1,40,53,<중립> 저는 조용하고 사적인 장소도 좋아하고 사람들로 붐비고 떠들썩한 장소도 좋아...
2,1,40,56,<그렇다> 저는 규칙을 잘 지키고 매뉴얼 대로 일하는 사람입니다. 그래서 데이터 라...
3,1,40,60,<그렇다> 저는 항상 긍정적인 사고방식을 가지고 살려고 노력하고 있습니다. 이유는 ...
4,1,30,51,<중립> 혼자서 일하는 것도 좋고 함께 일하는 것도 모두 좋은데 같이 의논하는 일도...
...,...,...,...,...
2875,1,20,59,<아니다> 마감 기한은 웬만해서는 지키려고 하는 편이에요. 거의 대부분 지각하지 않아요.
2876,0,20,54,<중립> 저는 상대방의 감정을 바로는 아니지만 어느 정도 알아차릴 수는 있습니다. ...
2877,1,30,50,<그렇다> 상대방이 높게 평가할 수 롤 작은 실수도 크게 보일 수 있기 때문이에요....
2878,1,30,50,<아니다> 상대방이 나를 높게 평가하면 더 잘하면 된다고 생각하기 때문에 상대방이 ...


In [87]:
# load pretrained model
def get_model():
    # * Model          | Tokenizer          | Pretrained weights shortcut
    # MODEL=(DistilBertModel, DistilBertTokenizer, 'distilbert-base-uncased')
    tokenizer = BertTokenizerFast.from_pretrained("klue/bert-base")
    model = BertModel.from_pretrained("klue/bert-base")
    n_hl = model.config.num_hidden_layers
    # embed_dim = model.config.embedding_size
    # return model, tokenizer, n_hl, embed_dim
    return model, tokenizer, n_hl

In [88]:
# model, tokenizer, n_hl, embed_dim = get_model()
model, tokenizer, n_hl = get_model()

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "a

In [89]:
class MyMapDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        data = {k:v[idx] for k,v in self.data.items()}
        return data

In [90]:
# Concatenate the 'Question', 'Answer', 'Age' and 'Gender' columns

texts = [str([df['Q_number'][i]]) + ' ' + str(df['Answer'][i]) + ' ' + str(df['Age'][i]) + ' ' + str(df['Gender'][i]) for i in range(len(df))]
# tensor = tokenizer(df['Answer'].to_list(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)
tensor = tokenizer(texts, max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)
ds = MyMapDataset(tensor)
dl = DataLoader(ds, batch_size=32, shuffle=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [91]:
def forward(model, dl, device=0):
    pooled = []
    hidden = []
    model.cuda(device)
    model.eval()
    for data in dl:
        data = {k:v.cuda(device) for k,v in data.items()}
        with torch.no_grad():
            output = model(**data, output_hidden_states=True)
        p, h = output.pooler_output, output.hidden_states
        pooled.append(p) # pooler output
        hidden.append(h[-1][:,0,:]) # only [CLS] token embedding 
    return torch.cat(pooled), torch.cat(hidden)

In [92]:
result = forward(model, dl, device=0)

In [93]:
result[0].shape, result[1].shape

(torch.Size([2880, 768]), torch.Size([2880, 768]))

In [94]:
MBTI_epoch_list

[185, 193, 116, 739]

In [95]:
ie_path = f'/content/drive/MyDrive/qia-hackathon2023/data/{now}/ckpt/IE/epoch_{MBTI_epoch_list[0]}.pt'
sn_path = f'/content/drive/MyDrive/qia-hackathon2023/data/{now}/ckpt/SN/epoch_{MBTI_epoch_list[1]}.pt'
tf_path = f'/content/drive/MyDrive/qia-hackathon2023/data/{now}/ckpt/TF/epoch_{MBTI_epoch_list[2]}.pt'
jp_path = f'/content/drive/MyDrive/qia-hackathon2023/data/{now}/ckpt/JP/epoch_{MBTI_epoch_list[3]}.pt'
path = [ie_path, sn_path, tf_path, jp_path]

In [96]:
# model = torch.load(ie_path, map_location='cpu')
path

['/content/drive/MyDrive/qia-hackathon2023/data/05_24_17/ckpt/IE/epoch_185.pt',
 '/content/drive/MyDrive/qia-hackathon2023/data/05_24_17/ckpt/SN/epoch_193.pt',
 '/content/drive/MyDrive/qia-hackathon2023/data/05_24_17/ckpt/TF/epoch_116.pt',
 '/content/drive/MyDrive/qia-hackathon2023/data/05_24_17/ckpt/JP/epoch_739.pt']

In [97]:
def main(path):
    model = torch.load(path, map_location='cpu')
    model.eval()
    model = model.cuda(0)
    with torch.no_grad():
        output = model(result[0].cuda(0))
        output_prob = torch.softmax(output, dim=1) # get probability
        output_prob = output_prob.cpu().numpy()
        # output = output.argmax(axis=1)
    
    return output_prob

In [98]:
ie_prob = main(ie_path)
sn_prob = main(sn_path)
tf_prob = main(tf_path)
jp_prob = main(jp_path)

In [99]:
test = pd.DataFrame({'I/E': ie_prob[:, 1], 
                    'S/N': sn_prob[:, 1], 
                    'T/F': tf_prob[:, 1], 
                    'J/P': jp_prob[:, 1]})

In [100]:
test.index = np.arange(1, len(test)+1)
test = test.rename_axis(columns="idx")

In [101]:
test.head()

idx,I/E,S/N,T/F,J/P
1,0.498507,0.553861,0.721876,0.029481
2,0.534697,0.119817,0.672349,0.783092
3,0.787638,0.618457,0.832309,0.623106
4,0.679978,0.499594,0.747264,0.057612
5,0.541575,0.365030,0.550701,0.970848


In [102]:
test.to_csv(f'/content/drive/MyDrive/qia-hackathon2023/data/{now}/basemodel_result.csv', index_label = 'idx')